# Goal of the project

The goal of the project is to have a Arduino take and process an image taken from the camera of the arduino, then image to send to the laptop for a NN to identify the digit that it was given to it

## How will this be achived?
Firstly, I will use Pytorch, A well known Machine learning library to make and train the model. Then using pygame and make a simple Paint2D interface for the users to draw their digit. Then with Pyserial I will use a serial connection to the arduino to indicate which class the AI thinks it is and then output it on the LEDs for the user to see.

In [1]:
import torch

In [12]:
# Now we can get the MNIST dataset from the torchvision class and import toTensor so they become Pytorch Tensors
from torchvision import datasets
from torchvision import transforms

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

Data = datasets.MNIST(root="data", download=False, train=True, transform=transform)

Now we have all the data we need to start training the model.

But first we should split the data into Training (70%), Testing(20%) and Validation(10%) Sets to ensure good learning

In [13]:
from torch.utils.data import random_split

TRAINING_SIZE   = int(len(Data) * 0.7)
TESTING_SIZE    = int(len(Data) * 0.2)
VALIDATION_SIZE = int(len(Data) * 0.1)

TrainingData, TestingData, ValidationData = random_split(Data, [TRAINING_SIZE, TESTING_SIZE, VALIDATION_SIZE])

Now we can put them in a Dataloader.

In [14]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64

TrainLoader      = DataLoader(TrainingData,   batch_size=BATCH_SIZE, shuffle=True)
TestingLoader    = DataLoader(TestingData,    batch_size=BATCH_SIZE, shuffle=True)
ValidationLoader = DataLoader(ValidationData, batch_size=BATCH_SIZE, shuffle=True)

Now we have everything ready and can start building the model

In [ ]:
import torch
import torch.nn as nn

# Define the Model Architecture
class NumberIdentifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.Identifier = nn.Sequential(
            # Our Input Features are 28 x 28 so 784 
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10) # We have 10 digits to identify from 0 -> 9 so we have ten output classes
        )

    def forward(self, x):
        # Flatten x so it isn't a 28 x 28 and a 1D 784 image
        x = x.view(x.size(0), -1)
        return self.Identifier(x)
    
    def predict(self, x):
        x = x.view(x.size(0), -1)
        return torch.argmax(self.Identifier(x))


We have built our model it has 4 total layers(Two being hidden layers) and all layers have the ReLU activation function.

Now lets train it!

In [22]:
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR

ImageClassifier = NumberIdentifier() # Unfortunetly we don't have a GPU so we can't speed it up
Optimizer       = Adam(ImageClassifier.parameters(), lr=0.001)
CostFunction    = nn.CrossEntropyLoss()
Scheduler       = ExponentialLR(Optimizer, gamma=0.8) # To prevent overfitting the model we will use this to exponentially decrease the learning rate every epoch

In [ ]:
from tqdm import tqdm # Another library which just shows us % done of loops

# Training Loop
NUMEPOCHS = 4

for i in range(NUMEPOCHS):
    # Training Loop
    ImageClassifier.train()
    TrainingLoss     = 0
    TrainingAccuracy = 0

    for batch in tqdm(TrainLoader, desc="Training"):
        x, y = batch
        
        # Do the forwards pass on our model to find out the error
        y_hat = ImageClassifier(x)

        # Calculate the cost
        Cost = CostFunction(y_hat, y)

        # Backwards pass
        Optimizer.zero_grad()
        Cost.backward()
        Optimizer.step()

        # Record Loss and Accuracy
        TrainingLoss += Cost.item()
        TrainingAccuracy += (y_hat.argmax(1) == y).type(torch.float).sum().item()
    
    TrainingAccuracy /= len(TrainLoader) * BATCH_SIZE
    TrainingLoss     /= len(TrainLoader)

    print(f"Epoch {i + 1}/{NUMEPOCHS}, Avg. Training Loss: {(TrainingLoss / len(TrainLoader)):.4f}, Training Accuracy {TrainingAccuracy * 100}%")

    # For Validation loop it's pretty much the same
    ImageClassifier.eval()
    ValidationLoss     = 0
    ValidationAccuracy = 0

    with torch.no_grad():
        for batch in tqdm(ValidationLoader, desc="Validating"):
            x, y = batch

            # Do the forwards pass on our model to find out the error
            y_hat = ImageClassifier(x)

            # Calculate the cost
            ValidationCost = CostFunction(y_hat, y)

            # Record loss and Accuracy
            ValidationLoss += ValidationCost.item()
            ValidationAccuracy += (y_hat.argmax(1) == y).type(torch.float).sum().item()
    
    ValidationAccuracy /= len(ValidationLoader) * BATCH_SIZE
    ValidationLoss     /= len(ValidationLoader)

    print(f"Epoch {i + 1}/{NUMEPOCHS}, Avg. Validation Loss: {(ValidationLoss):.4f}, Validation Accuracy { ValidationAccuracy * 100}%")

    # Adjust Learning Rate
    Scheduler.step()

Now we trained the AI lets test it to see how well it performs on unseen data.

In [8]:
# Mostly the same thing with the validation loop
ImageClassifier.eval()

TestingLoss = 0
CorrectPredictions = 0

with torch.no_grad():
    TestingAccuracy = 0
    for batch in tqdm(TestingLoader, desc="Testing"):
        x, y = batch

        # Get the forward passes
        y_hat = ImageClassifier(x)

        # Calculate the cost
        TestingLoss = CostFunction(y_hat, y)

        # Record loss and Accuracy
        TestingLoss += TestingLoss.item()
        CorrectPredictions += (y_hat.argmax(1) == y).type(torch.float).sum().item()

CorrectPredictions /= len(TestingLoader) * BATCH_SIZE
print(f"\n Testing Complete! Average Testing Loss: {(TestingAccuracy / len(TestingLoader))}, Accuracy: {CorrectPredictions * 100}%")

Testing: 100%|██████████| 188/188 [00:01<00:00, 139.18it/s]


 Testing Complete! Average Testing Loss: 0.0, Accuracy: 97.00797872340425%


Now Lets save the model so we don't have to Train it every time we run the AI

In [10]:
torch.save(ImageClassifier.state_dict(), 'TrainedModel/ImageClassifier_97') # The Number at the end indicates its accuracy

In [ ]:
import torch

# Load the model
model = torch.load('TrainedModel/ImageClassifier_97')
model.eval()

In [ ]:
import matplotlib.pyplot as plt
import random

TestingData    = iter(TestingLoader)
Images, Labels = next(TestingData)

Index = random.randint(0, BATCH_SIZE - 1)

RandImage      = Images[Index]
RandImageIndex = Labels[Index]

Prediction = model.predict(RandImage)

print(f"The AI Prediction: {Prediction.item()}")
print(f"Actual Class:      {RandImageIndex.item()}")

plt.imshow(RandImage.permute(1, 2, 0))
plt.axis('off')
plt.show()